# Spatial aggregation by level

This notebook allows to compile the results of the person exposed building exposed and cost platform on a larger scale (nuts2 nuts1)

## gives the results on a larger scale

### install the libraries

In [ ]:
import pandas as pd
import os
from urllib.parse import urlparse
import requests

What information are you looking for?

In [ ]:
value_type = "population"# The possible values are population, number or cost
nuts_value = "FRL0"
year = "2100"# The possible values are 2010 , 2030, 2050, 2100
rp="100"# The possible values are static, 1, 100 or 1000
defense="HIGH_DEFENDED" # The possible values are HIGH_DEFENDED, LOW_DEFENDED or UNDEFENDED
ssp="SSP126" # The possible values are SSP126, SSP245 or SSP585

In [ ]:
def validate_inputs(year, rp, defense, ssp):
    valid_years = {"2010", "2030", "2050", "2100"}
    valid_rps = {"static", "1", "100", "1000"}
    valid_defenses = {"HIGH_DEFENDED", "LOW_DEFENDED", "UNDEFENDED"}
    valid_ssps = {"SSP126", "SSP245", "SSP585"}
    errors = []
    if year not in valid_years:
        errors.append(f"Invalid year: {year}. Must be one of {sorted(valid_years)}.")
    if rp not in valid_rps:
        errors.append(f"Invalid return period (rp): {rp}. Must be one of {sorted(valid_rps)}.")
    if defense not in valid_defenses:
        errors.append(f"Invalid defense level: {defense}. Must be one of {sorted(valid_defenses)}.")
    if ssp not in valid_ssps:
        errors.append(f"Invalid SSP scenario: {ssp}. Must be one of {sorted(valid_ssps)}.")
    if errors:
        raise ValueError("\n".join(errors))
validate_inputs(year, rp, defense, ssp)

connection to the catalog and download of the file

In [ ]:

defense=defense+"_MAPS"

if value_type == "population":
    path="https://storage.googleapis.com/coclico-data-public/coclico/pp_stats/map_stats/"+defense+"/"+rp+"/"+ssp+"/pop_stats_"+defense+"_"+rp+"_"+ssp+"_"+year+".parquet"
    local_path = "CoCliCo_data/people.parquet"
    response = requests.get(path)
    with open(local_path, "wb") as f:
        f.write(response.content)
    colum="abs_affected"
elif value_type == "number":
    path="https://storage.googleapis.com/coclico-data-public/coclico/be_stats/map_stats/"+defense+"/"+rp+"/"+ssp+"/be_stats_"+defense+"_"+rp+"_"+ssp+"_"+year+".parquet"
    local_path = "CoCliCo_data/number.parquet"
    response = requests.get(path)
    with open(local_path, "wb") as f:
        f.write(response.content)  
    colum="abs_affected"  
       
elif value_type == "cost":
    path = "https://storage.googleapis.com/coclico-data-public/coclico/bc_stats/bc_stats.parquet"
    local_path = "CoCliCo_data/cost.parquet"
    response = requests.get(path)
    with open(local_path, "wb") as f:
        f.write(response.content)
    colum=defense+"\\"+rp+"\\"+ssp+"\\"+year+"\\buildings"
else:
    raise ValueError(f"Unknown value_type '{value_type}'")


defines what scale is requested

In [ ]:
def detect_scale(code, df):
    if code in df['CNTR_CODE'].values:
        return 'CNTR_CODE'
    elif code in df['nuts_2'].values:
        return 'nuts_2'
    elif code in df['nuts_3'].values:
        return 'nuts_3'
    else:
        return 'unknown'


In [ ]:
df=pd.read_parquet(local_path)
scale=detect_scale(nuts_value,df)
filtered = df[df[scale] == nuts_value]
summary = filtered.groupby(scale)[colum].sum().reset_index()

print(summary)